# Extract all link files from http://web.mta.info/developers/turnstile.html

In [31]:
from bs4 import BeautifulSoup, SoupStrainer
import urllib.request as urllib2
from urllib.request import urlopen
import re
from os.path import exists,getsize
import os
from socket import timeout
from urllib.error import HTTPError, URLError

retries = 0

def download_all_files():
    global retries
    files_to_download = []
    html_page = urllib2.urlopen("http://web.mta.info/developers/turnstile.html")
    soup = BeautifulSoup(html_page)
    all_Data_Links = soup.findAll('a' ,attrs={'href': re.compile("^data")})
    for link in all_Data_Links:
        #print(link['href'].split("/"))
        fname = link['href'].split("/")[3]
        #check if file already exists or less than 20 Mega in size(partial file)
        path_to_file = "D:\\Proj\\1\\"+str(fname)
        if not exists(path_to_file) or getsize(path_to_file) <20*10**6:
            file = open(path_to_file,"wb")
            try:
                url = urllib2.urlopen('http://web.mta.info/developers/'+link['href'],timeout=120)
                #reset retries upon sucessful download
                print(fname,' Downloaded successfully')
                retries =0
            except (HTTPError, URLError) as error:
                print('Data of %s not retrieved because %s\nURL: %s', name, error, url)
            except timeout:
                print('socket timed out - URL %s', url)
                
                
                retries +=1
                if retries <4:
                    download_all_files()
                    break
                else:
                    print("Max retries reached")
                
            for line in url:
                #print(type(line))
                file.write(line + b'\n')
            file.close()
        #else:
            #print("file "+fname+" skipped")
        # add each link to a list
    #files_to_download.append('http://web.mta.info/developers/'+link['href'])


download_all_files()

turnstile_141206.txt  Downloaded successfully
turnstile_141129.txt  Downloaded successfully
turnstile_141011.txt  Downloaded successfully
turnstile_141004.txt  Downloaded successfully


KeyboardInterrupt: 

In [7]:
files_to_download = []
html_page = urllib2.urlopen("http://web.mta.info/developers/turnstile.html")
soup = BeautifulSoup(html_page)
all_Data_Links = soup.findAll('a' ,attrs={'href': re.compile("^data")})

In [8]:
all_Data_Links

[<a href="data/nyct/turnstile/turnstile_210918.txt">Saturday, September 18, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210911.txt">Saturday, September 11, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210904.txt">Saturday, September 04, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210828.txt">Saturday, August 28, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210821.txt">Saturday, August 21, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210814.txt">Saturday, August 14, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210807.txt">Saturday, August 07, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210731.txt">Saturday, July 31, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210724.txt">Saturday, July 24, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210717.txt">Saturday, July 17, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210710.txt">Saturday, July 10, 2021</a>,
 <a href="data/nyct/turnstile/turnstile_210703.txt">Saturday, July 03, 2021</a>,
 <a h

In [32]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import pyodbc
server = 'LAPTOP-88OFGUMN\SQLEXPRESS' 
database = 'T5_Proj1' 
username = 'sa' 
password = '102030' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

onlyfiles = [f for f in listdir("D:\\Proj\\1") if isfile(join("D:\\Proj\\1", f))]

for afile in onlyfiles:
    file_url = "D:\\Proj\\1\\"+afile
    df = pd.read_csv(file_url)
    

#col_names = ['C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC','ENTRIES','EXITS']
def get_data(week_nums):
    #url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    #df = pd.DataFrame()
    #print(df.head())
    dfs =[]
    for week_num in week_nums:
        file_url = week_num
        dfs.append(pd.read_csv("D:\\proj\\1\\"+week_num))
    return pd.concat(dfs)

onlyfiles = [f for f in listdir("D:\\Proj\\1") if isfile(join("D:\\Proj\\1", f))]
#week_nums = [160903, 160910, 160917]
turnstiles_df = get_data(onlyfiles[0:4])


In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import pyodbc
import time

server = 'LAPTOP-88OFGUMN\SQLEXPRESS' 
database = 'T5_Proj1' 
username = 'sa' 
password = '102030' 
#cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
#cursor = cnxn.cursor()

onlyfiles = [f for f in listdir("D:\\Proj\\1") if isfile(join("D:\\Proj\\1", f))]
def Import_CSV(files_path):
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()

    for afile in files_path:
        start = time.time()
        file_url = "D:\\Proj\\1\\"+afile
        print(file_url)
        df = pd.read_csv(file_url)
        #trim white spaces in column header
        df.columns=df.columns.str.strip()
        i =0
        for index, row in df.iterrows():
            cursor.execute("Insert into MTA2 (C_A,Unit,SCP,Station,LineName,Division,Date_str,Time_str,Description,Entries,Exits) \
            values (?,?,?,?,?,?,?,?,?,?,?)",(row['C/A'],row['UNIT'],row['SCP'],row['STATION'],row['LINENAME'],row['DIVISION'],row['DATE'],row['TIME'],row['DESC'],row['ENTRIES'],row['EXITS'] ))
            cnxn.commit()
            i  +=1
        elapsed = time.time() - start

        print(afile," has been added with "+str(i)+ " rows")
        print("took "+str(elapsed)+ " seconds to add records")
    cnxn.close()


In [3]:
## multi threading 
import _thread
import time

onlyfiles = [f for f in listdir("D:\\Proj\\1") if isfile(join("D:\\Proj\\1", f))]

# Create two threads as follows
#try:
_thread.start_new_thread( Import_CSV, (onlyfiles[0:100] ,) )
_thread.start_new_thread( Import_CSV, (onlyfiles[100:200],) )
_thread.start_new_thread( Import_CSV, (onlyfiles[200:300],) )
_thread.start_new_thread( Import_CSV, (onlyfiles[300:],) )
#except:
 #  print("Error: unable to start thread")


23832

D:\Proj\1\turnstile_141018.txt
D:\Proj\1\turnstile_200718.txt
D:\Proj\1\turnstile_180818.txt
D:\Proj\1\turnstile_160917.txt
turnstile_141018.txt  has been added with 184959 rows
took 255.74831199645996 to add records
D:\Proj\1\turnstile_141025.txt
turnstile_160917.txt  has been added with 192878 rows
took 264.2122881412506 to add records
D:\Proj\1\turnstile_160924.txt
turnstile_180818.txt  has been added with 195261 rows
took 269.1999263763428 to add records
D:\Proj\1\turnstile_180825.txt
turnstile_200718.txt  has been added with 206262 rows
took 284.1349503993988 to add records
D:\Proj\1\turnstile_200725.txt
turnstile_141025.txt  has been added with 193065 rows
took 251.09472370147705 to add records
D:\Proj\1\turnstile_141101.txt
turnstile_160924.txt  has been added with 192678 rows
took 251.16578722000122 to add records
D:\Proj\1\turnstile_161001.txt
turnstile_180825.txt  has been added with 197380 rows
took 257.4152090549469 to add records
D:\Proj\1\turnstile_180901.txt
turnstile_20

Exception ignored in thread started by: <function Import_CSV at 0x00000212364034C0>
Traceback (most recent call last):
  File "<ipython-input-2-0b1e4b4d8cfe>", line 28, in Import_CSV
pyodbc.ProgrammingError: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 13 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')


turnstile_161029.txt  has been added with 192515 rows
took 236.98278307914734 to add records
D:\Proj\1\turnstile_161105.txt
turnstile_180929.txt  has been added with 200048 rows
took 242.62745213508606 to add records
D:\Proj\1\turnstile_181006.txt
turnstile_200829.txt  has been added with 217832 rows
took 257.42301774024963 to add records
D:\Proj\1\turnstile_200905.txt
turnstile_161105.txt  has been added with 191622 rows
took 220.04656147956848 to add records
D:\Proj\1\turnstile_161112.txt
turnstile_181006.txt  has been added with 198733 rows
took 227.03728675842285 to add records
D:\Proj\1\turnstile_181013.txt
turnstile_200905.txt  has been added with 208051 rows
took 235.58697867393494 to add records
D:\Proj\1\turnstile_200912.txt
turnstile_161112.txt  has been added with 194409 rows
took 220.08661699295044 to add records
D:\Proj\1\turnstile_161119.txt
turnstile_181013.txt  has been added with 199443 rows
took 221.9274547100067 to add records
D:\Proj\1\turnstile_181020.txt
turnstile

In [26]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("D:\\Proj\\1") if isfile(join("D:\\Proj\\1", f))]

In [1]:
onlyfiles

NameError: name 'onlyfiles' is not defined

In [29]:
import pyodbc
server = 'LAPTOP-88OFGUMN\SQLEXPRESS' 
database = 'T5_Proj1' 
username = 'sa' 
password = '102030' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
#for index, row in df.iterrows():
cursor.execute("Insert into MTA (C_A) values ('')", )
cnxn.commit()
cursor.close()

In [30]:
from sqlalchemy import create_engine 
import urllib
server = 'LAPTOP-88OFGUMN\SQLEXPRESS' 
database = 'T5_Proj1' 
username = 'sa' 
password = '102030' 

params = urllib.parse.quote_plus(
'DRIVER={ODBC Driver 17 for SQL Server};'+ 
'SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params) 

#Checking Connection 
connected = pd.io.sql._is_sqlalchemy_connectable(engine)

print(connected)

NameError: name 'pd' is not defined

In [29]:
print(dfs.head())
#df.to_sql('MTA3', con=engine, if_exists='append', index=False)


"""
if_exists: {'fail', 'replace', 'append'}, default 'fail'
     fail: If table exists, do nothing.
     replace: If table exists, drop it, recreate it, and insert data.
     append: If table exists, insert data. Create if does not exist.
"""

NameError: name 'dfs' is not defined

In [28]:
from datetime import datetime, timedelta


#d = datetime.strptime("2021-09-18",'%Y-%m-%d') - timedelta(days=7)

list_of_dates = [(datetime.strptime("2021-09-18",'%Y-%m-%d') - timedelta(days=i*7)).strftime('%y%m%d') for i in range(0,12)]
list_of_dates

['210918',
 '210911',
 '210904',
 '210828',
 '210821',
 '210814',
 '210807',
 '210731',
 '210724',
 '210717',
 '210710',
 '210703']